In [386]:
from sys import maxsize

import numpy as np
import itertools
import math
import matplotlib.pyplot as plt
import qiskit as qk
from qiskit import QuantumCircuit
#from qiskit import Aer

from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
#from qiskit.providers.aer import QasmSimulator
import qiskit_aer.noise as noise
from qiskit.circuit.library import UnitaryGate

#My imports
from coolingUnitary import CoolingUnitary
from mirrorProtocol import MirrorProtocol
from pairingPartnerAlgorithm import PairingPartnerAlgorithm
from minimalWorkProtocol import MinimalWorkProtocol
from utils import *

In [387]:
backendName = "ibm_osaka"

In [388]:
service = QiskitRuntimeService()
#sim_backend = service.backend(backendName)
qasm_sim = AerSimulator()

In [389]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi

""" qreg_q = QuantumRegister(3, 'q')
creg_c = ClassicalRegister(3, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.barrier()
circuit.x(qreg_q[0])
circuit.ccx(qreg_q[0], qreg_q[1], qreg_q[2])
circuit.x(qreg_q[0])
circuit.barrier()
circuit.x(qreg_q[0])
circuit.x(qreg_q[2])
circuit.ccx(qreg_q[0], qreg_q[2], qreg_q[1])
circuit.x(qreg_q[0])
circuit.x(qreg_q[2])
circuit.barrier()
circuit.x(qreg_q[1])
circuit.x(qreg_q[2])
circuit.ccx(qreg_q[2], qreg_q[1], qreg_q[0])
circuit.x(qreg_q[1])
circuit.x(qreg_q[2])
circuit.barrier()
circuit.x(qreg_q[0])
circuit.x(qreg_q[2])
circuit.ccx(qreg_q[0], qreg_q[2], qreg_q[1])
circuit.x(qreg_q[0])
circuit.x(qreg_q[2])
circuit.barrier()
circuit.x(qreg_q[0])
circuit.ccx(qreg_q[0], qreg_q[1], qreg_q[2])
circuit.x(qreg_q[0]) """
# @columns [1,1,1,2,3,4,5,5,5,6,6,7,8,8,9,9,9,10,10,11,12,12,13,13,13,14,14,15,16,16,17,17,17,18,19,20]

" qreg_q = QuantumRegister(3, 'q')\ncreg_c = ClassicalRegister(3, 'c')\ncircuit = QuantumCircuit(qreg_q, creg_c)\n\ncircuit.barrier()\ncircuit.x(qreg_q[0])\ncircuit.ccx(qreg_q[0], qreg_q[1], qreg_q[2])\ncircuit.x(qreg_q[0])\ncircuit.barrier()\ncircuit.x(qreg_q[0])\ncircuit.x(qreg_q[2])\ncircuit.ccx(qreg_q[0], qreg_q[2], qreg_q[1])\ncircuit.x(qreg_q[0])\ncircuit.x(qreg_q[2])\ncircuit.barrier()\ncircuit.x(qreg_q[1])\ncircuit.x(qreg_q[2])\ncircuit.ccx(qreg_q[2], qreg_q[1], qreg_q[0])\ncircuit.x(qreg_q[1])\ncircuit.x(qreg_q[2])\ncircuit.barrier()\ncircuit.x(qreg_q[0])\ncircuit.x(qreg_q[2])\ncircuit.ccx(qreg_q[0], qreg_q[2], qreg_q[1])\ncircuit.x(qreg_q[0])\ncircuit.x(qreg_q[2])\ncircuit.barrier()\ncircuit.x(qreg_q[0])\ncircuit.ccx(qreg_q[0], qreg_q[1], qreg_q[2])\ncircuit.x(qreg_q[0]) "

In [390]:
""" circuit.draw('mpl') """

" circuit.draw('mpl') "

In [412]:
def permutationToCircuit2(l):
    if(len(l) > 0):
        numQubit = len(l[0][0])
        qreg_q = QuantumRegister(numQubit, 'q')
        creg_c = ClassicalRegister(numQubit, 'c')
        finalCircuit = QuantumCircuit(qreg_q, creg_c)
        finalCircuit.barrier()
        for i in range(len(l)):
            #stateIn = l[i][0]
            stateIn = l[i][0][::-1]
            stack = []
            #for j in range(numQubit-1,-1,-1):
            for j in range(numQubit):
                circuit = QuantumCircuit(qreg_q)
                if(l[i][0][j] != l[i][1][j]):
                    
                    #Add X gates when the qubit is in the state 0 so all control are in the 1 state
                    for z in range(len(stateIn)):
                        if(z != j):
                            if(stateIn[z] == '0'):
                                circuit.x(z)

                    
                    #MCX Gate with target in the J position and control in the rest of the qubits
                    rangeOfMCX = list(range(0,j)) + list(range(j+1,numQubit))
                    #print(rangeOfMCX)
                    #print(l[i][0][j] + " " + l[i][1][j])
                    circuit.mcx(list(rangeOfMCX),j)

                    #Add X gates when the qubit is in the state 0 so all control are in the 1 state
                    for z in range(len(stateIn)):
                        if(z != j):
                            if(stateIn[z] == '0'):
                                circuit.x(z)

                    #Change the state in to match the state after applying the gate
                    if(stateIn[j] == '0'):
                        stateIn = stateIn[:j] + '1' + stateIn[j + 1:]
                    else:
                        stateIn = stateIn[:j] + '0' + stateIn[j + 1:]

                    circuit.barrier()

                    stack.append(circuit)
                    finalCircuit = finalCircuit.compose(circuit)

           
            #Using the stack we create the uncomputation circuit
            #We discard the first element since we don't need it for the uncompuation
            stack.pop()
            while(len(stack)):
                finalCircuit = finalCircuit.compose(stack.pop())

    return finalCircuit

In [547]:
def permutationToCircuit(l):
    if(len(l) > 0):
        numQubit = len(l[0][0])
        qreg_q = QuantumRegister(numQubit, 'q')
        creg_c = ClassicalRegister(numQubit, 'c')
        finalCircuit = QuantumCircuit(qreg_q, creg_c)
        finalCircuit.barrier()
        for i in range(len(l)):
            #stateIn = l[i][0]
            #print(l[i][0])
            stateIn = l[i][0][::-1]
            stack = []
            for j in range(numQubit-1,-1,-1):
                #The reason we use opposite is because how Qiskit handles the states
                #The qubit we want to cool down is the n-th one
                opposite = numQubit - j - 1            
                circuit = QuantumCircuit(qreg_q)
                if(l[i][0][j] != l[i][1][j]):
                    
                    #Add X gates when the qubit is in the state 0 so all control are in the 1 state
                    for z in range(len(stateIn)):
                        if(z != opposite):
                            if(stateIn[z] == '0'):
                                circuit.x(z)
                    
                    #MCX Gate with target in the (numQubit - j - 1) position and control in the rest of the qubits
                    rangeOfMCX = list(range(0,opposite)) + list(range(opposite + 1,numQubit))
                    #print(rangeOfMCX)
                    #print(l[i][0][j] + " " + l[i][1][j])
                    circuit.mcx(list(rangeOfMCX),opposite)

                    #Add X gates when the qubit is in the state 0 so all control are in the 1 state             
                    for z in range(len(stateIn)):
                        if(z != opposite):
                            if(stateIn[z] == '0'):
                                circuit.x(z)

                    #Change the state in to match the state after applying the gate              
                    if(stateIn[opposite] == '0'):
                        stateIn = stateIn[:opposite] + '1' + stateIn[opposite + 1:]
                    else:
                        stateIn = stateIn[:opposite] + '0' + stateIn[opposite + 1:]


                    circuit.barrier()

                    stack.append(circuit)
                    finalCircuit = finalCircuit.compose(circuit)
                    
                

           
            #Using the stack we create the uncomputation circuit
            #We discard the first element since we don't need it for the uncompuation
            stack.pop()
            while(len(stack)):
                finalCircuit = finalCircuit.compose(stack.pop())

    return finalCircuit

In [563]:
def testCircuit(circuit):
    n = circuit.num_qubits
    num_circs = 2**n
    circuits = []
    q_regs = QuantumRegister(n, 'q')
    c_regs = ClassicalRegister(n, 'c')
    #make circuits
    for i in range(num_circs):
        total_circ = QuantumCircuit(q_regs, c_regs)
        #prepare initial state
        bitstring = format(i,'b').zfill(n)
        idx = 0
        for bit in bitstring[::-1]:
        #for bit in bitstring:
            if (bit == '1'):
                total_circ.x(idx)
            idx += 1
        total_circ.barrier()
        #add cooling circuits
        #total_circ.compose(circuit, qubits=[q_regs[2],q_regs[1],q_regs[0]], inplace=True)
        total_circ = total_circ.compose(circuit)
        #total_circ.compose(cooling_circuit, qubits=[q_regs[0],q_regs[1],q_regs[2],q_regs[3],q_regs[4]], inplace=True)
        #add measurement
        total_circ.barrier()
        total_circ.measure(list(range(n)), c_regs)
        #total_circ.measure(0, c_regs)
        #transpiledCircuit = qk.transpile(total_circ, backend=sim_backend,optimization_level=3)
        circuits.append(total_circ)
    
    counts = []


    print(format(0).zfill(1 + num_circs // 10) + " | ",end="")
    for i in range(n-1,0,-1):
        print(i,end="")
    print(0)
    print("---------------------")
    for i in range(num_circs):
        result = qasm_sim.run(circuits[i],shots = 1).result()
        counts.append(result.get_counts())
        #print(i, list(counts[i])[0])
        if(format(i,'b').zfill(n) == list(counts[i])[0]):
            print(format(i).zfill(1 + num_circs // 10) + " | " + format(i,'b').zfill(n) + " --> " + str(list(counts[i])[0]))
        else:
            print(format(i).zfill(1 + num_circs // 10) + " | " + format(i,'b').zfill(n) + " --> " + str(list(counts[i])[0]) + " (*)")
    #return circuits

In [568]:
l = [["1000","0001"]]

c = permutationToCircuit(l)

c.draw(fold=maxsize)

░      ┌───┐      ░                 ░      ┌───┐      ░ 
q_0: ─░──────┤ X ├──────░────────■────────░──────┤ X ├──────░─
      ░ ┌───┐└─┬─┘┌───┐ ░ ┌───┐  │  ┌───┐ ░ ┌───┐└─┬─┘┌───┐ ░ 
q_1: ─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─
      ░ ├───┤  │  ├───┤ ░ ├───┤  │  ├───┤ ░ ├───┤  │  ├───┤ ░ 
q_2: ─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─
      ░ └───┘  │  └───┘ ░ └───┘┌─┴─┐└───┘ ░ └───┘  │  └───┘ ░ 
q_3: ─░────────■────────░──────┤ X ├──────░────────■────────░─
      ░                 ░      └───┘      ░                 ░ 
c: 4/═════════════════════════════════════════════════════════

In [571]:
testCircuit(c)

#circs[0].draw()

00 | 3210
---------------------
00 | 0000 --> 0000
01 | 0001 --> 1000 (*)
02 | 0010 --> 0010
03 | 0011 --> 0011
04 | 0100 --> 0100
05 | 0101 --> 0101
06 | 0110 --> 0110
07 | 0111 --> 0111
08 | 1000 --> 0001 (*)
09 | 1001 --> 1001
10 | 1010 --> 1010
11 | 1011 --> 1011
12 | 1100 --> 1100
13 | 1101 --> 1101
14 | 1110 --> 1110
15 | 1111 --> 1111


In [570]:
n = 3
a = MinimalWorkProtocol(n,0.01)
#checkUnitary(a)
cooling_circuit = QuantumCircuit(n)
cooling_circuit.append(UnitaryGate(a),range(n))

testCircuit(cooling_circuit)


0 | 210
---------------------
0 | 000 --> 000
1 | 001 --> 001
2 | 010 --> 010
3 | 011 --> 100 (*)
4 | 100 --> 011 (*)
5 | 101 --> 101
6 | 110 --> 110
7 | 111 --> 111
